In [2]:
#from code.utils import *
import pandas as pd
from io import StringIO
import os
import requests
import requests
from bs4 import BeautifulSoup as soup
from typing import List
from typing import Dict
import numpy as np
import unicodedata
import basedosdados as bd


In [56]:
import pandas as pd
from io import StringIO
import requests
import unicodedata
from typing import Dict


#---- mudar nome das colunas ----#

def change_column_name(
    url_architecture: str
)-> Dict[str, str]:
    
    """Essa função recebe como input uma string com link para uma tabela de arquitetura
    e retorna um dicionário com os nomes das colunas originais e os nomes das colunas 
    padronizados
    Returns:
        dict: com chaves sendo os nomes originais e valores sendo os nomes padronizados
    """
    # Converte a URL de edição para um link de exportação em formato csv
    url = url_architecture.replace(
        "edit#gid=",
        "export?format=csv&gid="
    )
    
    Coloca a arquitetura em um dataframe
    df_architecture = pd.read_csv(
        StringIO(requests.get(url, timeout=10).content.decode("utf-8"))
    )
        
    # Cria um dicionário de nomes de colunas e tipos de dados a partir do dataframe df_architecture
    column_name_dict = dict(
        zip(df_architecture['original_name'],df_architecture['name'])
    )
    
    # Retorna o dicionário 

    return column_name_dict

#---- mudar tipos de dados ----#

def change_dtypes(
    url_architecture: str
)-> Dict[str, str]:
    
    """Essa função recebe como input uma string com link para uma tabela de arquitetura
    e retorna um dicionário com os nomes das colunas originais e os nomes das colunas 
    padronizados
    Returns:
        dict: com chaves sendo os nomes originais e valores sendo os nomes padronizados
    """
    # Converte a URL de edição para um link de exportação em formato csv
    url = url_architecture.replace(
        "edit#gid=",
        "export?format=csv&gid="
    )
    
    Coloca a arquitetura em um dataframe
    df_architecture = pd.read_csv(
        StringIO(requests.get(url, timeout=10).content.decode("utf-8"))
    )
    

    # Cria um dicionário de nomes de colunas e tipos de dados a partir do dataframe df_architecture
    column_name_dict = dict(
        zip(df_architecture['original_name'],df_architecture['bigquery_type'])
    )

    #O pandas não consegue ler ints que tenham NAs
    #Para contornar isso e não adicionar 0. ao final de cada número,
    #optei por converter  todos os inteiros para string
    
    #loop para padronizar os tipos de dados e converter in para string
    for key, value in column_name_dict.items():
        if value == 'string':
            column_name_dict[key] = str
        elif value == 'int64':
            column_name_dict[key] = str
        elif value == 'float64':
            column_name_dict[key] = float

    return column_name_dict


#---- remover acentos e caracteres especiais ----#
def remove_accents(
    input_str
):
    """Essa função é aplicada com método apply em uma coluna de um dataframe para remover
    acentos e caracteres especiais de uma string. Exemplo de uso

    df[x].apply(remove_accents)

    Args:
        input_str (pd.Series): Uma coluna com strings

    Returns:
        pd.Series : coluna com strings sema acentos e caracteres especiais
    """    
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])


In [ ]:
#municipio
dir_path  = r'\br_mdr_snis\input\municipio'
files = os.listdir(dir_path)

lista_dfs = []

print('baixando dicionario da tabela de arquitetura para renomear colunas')
dicionario = change_column_name(
    url_architecture = 'https://docs.google.com/spreadsheets/d/1OA4mSAo99kY7vN4hf3-n22fAhbcXMPuyF1uxGBIo3UE/edit?gid=0#gid=0'
)

print('baixando tipos de dados da tabela de arquitetura para importar dados')
dicionario_dtypes = change_dtypes(
    url_architecture='https://docs.google.com/spreadsheets/d/1OA4mSAo99kY7vN4hf3-n22fAhbcXMPuyF1uxGBIo3UE/edit?gid=0#gid=0'
)

print('baixando diretorios de municípios da BD')
municipio = bd.read_table(
    dataset_id= 'br_bd_diretorios_brasil',
    table_id= 'municipio',
    billing_project_id= "pisagab-staging"
    )
municipio_dict = dict(zip(municipio.id_municipio_6, municipio.id_municipio))


for file in files:
    
    print(f'lendo arquivo {file}')
    df = pd.read_csv(
    dir_path + '\\' + file,
    sep=';',
    encoding= 'latin-1',
    decimal=',',
    thousands='.',
    dtype= dicionario_dtypes,
    )

    print('renomeando colunas')
    #rename
    df.rename(
        columns=dicionario, 
        inplace=True
    )

    print('adicionando id_municipio 7 dígitos')
    #add id_municipio 7 digits
    df['id_municipio'] = df['id_municipio'].map(municipio_dict)
    
    #check if there is any null value
    if df['id_municipio'].isna().sum() != 0:
        raise ValueError('id_municipio com valores nulos')
    
   
    print('dropando colunas')
    #delete cols
    df.drop(
        columns=[
        'Código do IBGE', 
        'Município',
        'Prestadores',
        'Serviços',
        'Natureza Jurídica',
        ], 
        inplace=True)
    
    print('tirando . de ints por precaução')
    
    strings_to_int = df.select_dtypes(include=['object']).columns
    
    for col in strings_to_int:
        df[col] = df[col].str.replace('.', '')
        print(f'col {col} formatada, . retirados')
    
    
    print('ordenando colunas')
    #redorder
    df = df[dicionario.values()]

    lista_dfs.append(df)

df = pd.concat(lista_dfs)

df.to_csv(
    r'\br_mdr_snis\output\municipio.csv',
    index=False,
    sep=',',
    encoding='utf-8',
    na_rep='',
)

In [ ]:
#prestador
dir_path  = r'\br_mdr_snis\input\prestador'
files = os.listdir(dir_path)

lista_dfs = []

print('baixando dicionario da tabela de arquitetura para renomear colunas')
dicionario = change_column_name(
    url_architecture = 'https://docs.google.com/spreadsheets/d/1spjI65YVI17mxtC1tmXaqMGk2pKAUPBJ5AqENKwB1Ys/edit?gid=0#gid=0'
)

print('baixando tipos de dados da tabela de arquitetura para importar dados')
dicionario_dtypes = change_dtypes(
    url_architecture='https://docs.google.com/spreadsheets/d/1spjI65YVI17mxtC1tmXaqMGk2pKAUPBJ5AqENKwB1Ys/edit?gid=0#gid=0'
)

print('baixando diretorios de municípios da BD')
municipio = bd.read_table(
    dataset_id= 'br_bd_diretorios_brasil',
    table_id= 'municipio',
    billing_project_id= "pisagab-staging"
    )

municipio_dict = dict(zip(municipio.id_municipio_6, municipio.id_municipio))


for file in files:
    
    print(f'lendo arquivo {file}')
    df = pd.read_csv(
    dir_path + '\\' + file,
    sep=';',
    encoding= 'latin-1',
    decimal=',',
    thousands='.',
    dtype= dicionario_dtypes,
    )

    print('renomeando colunas')
    #rename
    df.rename(
        columns=dicionario, 
        inplace=True
    )

    print('adicionando id_municipio 7 dígitos')
    #add id_municipio 7 digits
    df['id_municipio'] = df['id_municipio'].map(municipio_dict)
    
    #check if there is any null value
    if df['id_municipio'].isna().sum() != 0:
        raise ValueError('id_municipio com valores nulos')
    
    print('dropando colunas')
    #delete cols
    df.drop(
        columns=[
        'Município',
        ], 
        inplace=True)
   
    print('tirando . de ints por precaução')
    strings_to_int = df.select_dtypes(include=['object']).columns
    
    for col in strings_to_int:
        df[col] = df[col].str.replace('.', '')
        print(f'col {col} formatada, . retirados')
    
    
    print('removendo acentos')
    latin_list = [
        'prestador',
        'natureza_juridica',
        'tipo_servico',
        'local_atendimento_agua',
        'local_atendimento_esgoto',
    ]

    for col in latin_list:
        df[col] = df[col].apply(remove_accents)


    #insert id_natureza_juridica

    print('ordenando colunas')
    #redorder
    df = df[dicionario.values()]

    lista_dfs.append(df)

df = pd.concat(lista_dfs)

df.to_csv(
    r'\br_mdr_snis\output\prestador.csv',
    index=False,
    sep=',',
    encoding='utf-8',
    na_rep='',
)